<a href="https://colab.research.google.com/github/majorquev/DBDC_202409_Programacion_en_R_para_ciencia_de_datos/blob/main/Clases/C8/Clase8_1_Testeo_de_normalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte 7: Normalidad
## Análisis de un caso

**Autor:** Miguel Jorquera Viguera.<br>
**E-mail:** majorquev@gmail.com<br>
**Ayudante:** Ángel Ubilla Vera<br>
**E-mail ayudante:** rumorale1@gmail.com

En esta seción usted generará un breve anáisis descriptivo del comportamiento de las acciones de la empresa Mc'Donalds. Para ello ustede deberá instalar la ibrería `quantmod`, la cual disponibilizará funciones con las que se podrá acceder de manera sencilla a diferentes activos financieros y sus series históricas.


In [ ]:
library(quantmod) # debe instalarla de ser necesario
library(ggplot2)
library(dplyr)

La siguiente celda de código le permitirá cargar la serie histórica desde enero de 2007 hasta la fecha (día anterior). Dicha serie contiene la información del volumen, el valor de apertura, cierre, y cierre ajustado, en dólares.


In [ ]:
serie <- getSymbols("MCD", auto.assign = FALSE)
class(serie)
head(serie)

In [ ]:
tail(serie)

Note que este objeto es de la clase _xts_ y _zoo_, idóneas para trabajar con series de tiempo.
Algunas ventajas de trabajar con objetos de esta clase:
 * Permite visualizar de manera rápida un conjunto de series.
 * Varios modelos implementados en R y sus paquetes,está programados para manipular series de tiempo en este formato.

Si bien el objetivo no es ahondar en la teoría de series de tiempo, R ofrece algunas funcionalidades como las siguientes.

### Graficar

In [ ]:
options(repr.plot.width=12, repr.plot.height=6)
#require(TTR)
chartSeries(serie)

### Ajustar un modelo y generar predicciones
Existen diversas maneras de modelar una serie de tiempo, por ejemplo mediante modelos arima, garch, entre otros. Otra alternativa es a través de suavizados exponenciales, los cuales permiten modelar de manera adtiva o multiplicativa elementos como el error, la tendencia y la estacionalidad en una serie de tiempo (ETS, por los componentes Error, Trend, y Seasonality).

El objetivo de este ejemplo no es entender la teoría que hay detrás ni tampoco evaluar la pertinenia o calidad del modelo ajustado, si no que mostrar potecniales usos de R en cuánto al manejo de series de tiempo. Para más detalles pueden consultar las referencias del curso.

In [ ]:
mcd_close <- Cl(serie) # se extrae la columna MCD.Close
mod_t <- forecast::ets(mcd_close) # se ajustan de manera automatica un suavizado exponencial.
plot(forecast::forecast(mod_t,360)) # se genearn predicciones con un horizonte futuro de 360 días y se grafican dichos valores con sus respectivas bandas de confianza.

### Descomponer una serie
Una serie de tiempo posee elementos caraterísticos comola tendencia, estacionalidad y componente de error. En R existen funciones que permiten generar dichas descomposiciones de manera automática.
Por ejemplo, la data co2 contiene los niveles de c02 expresados en partes por millon (ppm) and reportados en la fracción de escala nanométrica preliminar del 1997

In [ ]:
plot(co2)
plot(decompose(co2))

## Análisis del valor ajustado
### Extracción de indicadores
Extraemos los indicadores asociados al activo MCD.

In [ ]:
serie_df <- data.frame(
                fecha = index(serie),
                coredata(serie)
            )
head(serie_df)

### Visualizaciones del valor ajustado
Podemos generar visuaizaciones del valor ajustado.   
  

In [ ]:

# a)
serie_df %>%
    ggplot() +
    aes(x = fecha, y = MCD.Adjusted) +
    geom_line() +
    geom_smooth()

# Se observa engenearl una tendencia al alza del valor de la acción de Mc'Donalds. Particularmente hacia fines del 20015 se puede apreciar una notable alza que se sostiene hacia mediados de 2019. Ya para el 2020 se evidencia una fuerte baja, probablemente debido  la pandemia.

In [ ]:
# b)


resumen <- serie_df %>%
    mutate(anio = lubridate::year(fecha)) %>%
    group_by(anio) %>%
    summarise(mean_adjusted = mean(MCD.Adjusted),
             sd_adjusted = sd(MCD.Adjusted)) %>%
    mutate(cv_adjusted = sd_adjusted/mean_adjusted)

resumen
# Se puede apreciar que en el año 2019 se obtuvo el mayor valor promedio de las acciones de Mc'Donalds. Siendo los años 2017, 2019 y 2020, con mayor variabilidad en el precio de la acción.
# Como complemento a la airmación anterior, se muestran los siguientes gráficos de caja.

serie_df %>%
    mutate(anio = lubridate::year(fecha)) %>%
    ggplot() +
    aes(x = as.factor(anio), y = MCD.Adjusted) +
    geom_boxplot() +
    labs()


### Distribución del valor ajustado

#### Histograma de frecuencias relativas y estimación de densidad dada por la lafunción `geom_density`


In [ ]:
serie_df %>%
    ggplot() +
    aes(x = MCD.Adjusted, y = ..density..) +
    geom_histogram() +
    geom_density(color = "red")

# Evidententemente la distribución del valor de la acciòn no es simétrica y además presenta dos picks (dos modas), por lo que no se observa una distribución normal.

#### Sólo observaciones comprendidas entre las fechas 2008-11-01 y 2008-10-31 inclusive


In [ ]:
serie_df %>%
    filter(fecha >= "2008-11-01" & fecha <="2009-10-31" ) %>%
    ggplot() +
    aes(x = MCD.Adjusted, y = ..density..) +
    geom_histogram() +
    geom_density(color = "red")



#### Añadiendo densidad normal con parámetros estimados  


In [ ]:
serie_df %>%
    filter(fecha >= "2008-11-01" & fecha <="2009-10-31" ) -> aux

aux %>%
    ggplot() +
    aes(x = MCD.Adjusted) +
    geom_histogram(aes(y = ..density..))+
    stat_function(fun = function(x){
        dnorm(x,mean = mean(aux$MCD.Adjusted), sd = sd(aux$MCD.Adjusted))
    }, col = "red") +
    geom_density(col = "blue")

## QQ-plots
Una manera gráfica de comparar distribuciones es a través de los gráficos quantile-quantile, también conocidos como `qq-plots`. Estos gráficos comparan los cuantiles entre dos distribuiones, de tal modo que si la gráfica entre ellos generan una "línea recta", se asume que ambas distribuciones coinciden. Este método es útil además, cuando se quiere comparar la distribución (desconocida) de una muestra contra una distribución  conocida,  como por ejemplo una normal,  donde se contrastan los cuantiles observados versus los teóricos. A modo de ejemplo, consideremos una muestra aleatoria de 1000 observaciones provenientes de una distribución normal con media 10 y desviación estándar. Podemos comparar los cuantiles observados con lo cuantiles teóricos de una normal estántar a través de la función qqnorm().

In [ ]:
set.seed(6669)
m1_norm <- rnorm(1000,mean= 10, sd =  5)
qqnorm(m1_norm) # contrasta contra los cuantiles de una normal estandar
qqline(m1_norm)

In [ ]:
m2_norm <- rnorm(2000,mean = 1, sd =  2)
# La función qqplot en cambio, compara los cuantiles observados enre dos muestras, por ejeplo, si comparamos m1_norm con m2_norm se observa el siguiente gráfico de cuantiles
qqplot(m1_norm,m2_norm)


Si en cambio contrastamos contra una normal,los cuantiles de una muestra que se be que no es normal, se observarán asimetrías en el gráfico de cuantiles. A modo de ejemplo, comparemos una muestra de una variable aleatoria con distribuciòn de T-student con 3 grados de libertad contra los cuantiles teóricos de una normal estándar.

In [ ]:
m3_t <- rt(1500,3)
qqnorm(m3_t)
qqline(m3_t)

In [ ]:
# Se puede observar una forma de "S", lo que indica que las colas de la muestra son más "pesadas" que las colas de lo normal. Es decir,  los primeros cuantiles de la muetra son menores que los pimeros cuantiles de una dist. normal, y los cuantiles superiores de la uestra sonmayores que los cuantiles superiores de la distribuciòn normal.

# Sólo recoradar que la distribuciòn t-student es similar a la normal pero con colas más "pesadas".(https://en.wikipedia.org/wiki/Student%27s_t-distribution)

# podemos Replicar el ejercicio con una distribución asimétrica como por ejemplo la distribuciòn gamma, donde se observa que los cuantiles iferiores y superiores son siempre mayores en la muestra gamma.
m4_gamma <- rgamma(1500,1,1)
hist(m4_gamma,freq = F) # histograma y densidad de una gamma con parámetros alpha = beta = 1
curve(dgamma(x,1,1), add = T ,col = "red")


qqnorm(m4_gamma)
qqline(m4_gamma)

### QQ-plot del valor ajustado
Consideramos el mismo período de observación entre 2008-11-01 y 2008-10-31 inclusive.

In [ ]:
qqnorm(aux$MCD.Adjusted)
qqline(aux$MCD.Adjusted) # Gráficamente se observa que los cuantiles observados se acercan bastantes a los teóricos de una distribuciòn normal, por o que pareciera ser pertinente el supuesto de normalidad.

## Test de normalidad
Si bien la inspecón visual es útil para generar ciertas conjeturas, es necesario contar con mecanismos más robustos para validar ciertos supuestos como lo es el caso de el de normalidad en una distribución. Una manera de probar si una mustra proviene de una distribución normal, es a través del test de shapiro-wilkk, cuya hipótesis nula establece que la muestra analizada proviene de una distribución normal (https://en.wikipedia.org/wiki/Shapiro%E2%80%93Wilk_test).

A modo de ejemplo, consideremos dos muestras, una normal(`m1_norm`) y otra no normal (la muestra gamma `m4_gamma` del ejemploo anterior). La función shapiro.test() evalúa la hipótesis nula, de que la muestra analizada tiene una distribución normal. Así, con un nivel de significancia del 5%, se rechaza la hipótesis de nornalidad para valores-p  menores a 0.05.



In [ ]:
shapiro.test(m1_norm) # no se rechaza la hipótesis nula, por lo que se puede establecer con un 5% de significancia que la muestra m1_norm proviene de una distribución normal.
shapiro.test(m4_gamma) # se rechaza la hipótesis nula, por lo que se puede establecer con un 5% de significancia que la muestra m4_gamma NO proviene de una distribución normal.

### ¿Cómo se interpreta el valor-p?
Es posible interpretar el valor-p como  la probabilidad de rechazar la hipótesis nula cuando esta es verdadera. Así, fijando un nivel del 5%, se tendrá que un test cuyo valor-p sea menor a 0.05, asegurará que en el 5% de las veces que se realice el experimento, se estará rechazando la hipótesis nula de manera incorrecta (vale decir cuando esta era verdadera).
Para entender este fenómeno, podemos generar la siguiente simulación:



In [ ]:
set.seed(2020)
valores_p <- c() # vector vacío

for (i in 1:3000){
    muestra <- rnorm(2000) # muestra aleatoria de 2000 obs de una normal con media 0 y sd igual a 1.
    test <- shapiro.test(muestra)
    valores_p <- c(valores_p,test$p.value)
}

In [ ]:
valores_p

In [ ]:
mean(valores_p < 0.05) # aproximadamente en el 5% de las simulaciones, se rechazó la hipótesis nulda de normaidad, sabiendo que l muestra era normal.

### Test de normalidad para el valor ajustado


In [ ]:
shapiro.test(aux$MCD.Adjusted) # el valor-p es mayor a 0.05, por  lo que no se rechaza la hipótesis de normalidad.

## Retornos y log-retornos de activos financieros
Usualmente es de interés modelar las variaciones porcentuales en el precio. A esta variación se le conoce como retorno (bruto), el cual se puede calcular como
$$ret=\dfrac{P_t}{P_{t-1}}$$
Donde $P_t$ corresponde al precio de la acción en el tiempo $t$. Una ventaja de modelar los retornos es que estos carecen de unidad de medida (son variaciones porcentuales), lo cual facilita el análisis y no introduce problemas de escala.

Por otro lado, en ciertas ocasiones las variaciones porcentuales de un instante de tiempo a otro pueden mostrar una alta variabilidad (desviación estándar,conocida como riesgo del activo). Así, una manera de estabilizar la varianza del activo en estudio, es analizar el logaritmo natural de los retornos brutos (log-returns).
$$lr_t = \log\left(\dfrac{P_t}{P_{t-1}}\right)$$

**NOTA:** Podemos utilizar la función `lag()` del package `dplyr`


In [ ]:
serie_df <- serie_df %>%
    mutate(lag = dplyr::lag(MCD.Adjusted)) %>%
    mutate(ret = MCD.Adjusted/lag,
           log_ret = log(MCD.Adjusted/lag ) )



In [ ]:
head(serie_df)

#### Visualización de los log-retornos

In [ ]:

serie_df %>%
    filter(fecha >= "2008-11-01" & fecha <="2009-10-31" )  %>%
    ggplot() +
    aes(x = fecha, y = MCD.Adjusted) +
    geom_line(color =  "red")

serie_df %>%
    filter(fecha >= "2008-11-01" & fecha <="2009-10-31" )  %>%
    ggplot() +
    aes(x = fecha, y = log_ret) +
    geom_line(color =  "blue")

# Visualmente se aprecia un comportamiento más "errátco" en los log-retornos, en términos de que no se observan tenencias marcadas, y aparentemente la volatilidad es más homogénea a través del tiempo. Por otra parte, la media de los log-retornos se encuentra en torno al cero.

### Normalidad de los log-retornos


In [ ]:
# histograma y densidades
aux2 <- serie_df %>%
    filter(fecha >= "2008-11-01" & fecha <="2009-10-31" )

aux2 %>%
    ggplot() +
    aes(x = log_ret) +
    geom_histogram(aes(y = ..density..))+
    stat_function(fun = function(x){
        dnorm(x,mean = mean(aux2$log_ret), sd = sd(aux2$log_ret))
    }, col = "red") +
    geom_density(col = "blue")

# qqplot
qqnorm(aux2$log_ret)
qqline(aux2$log_ret)

# test
shapiro.test(aux2$log_ret) # si bien se observan algunos posibles datos atípicos en el qqplot, el test de shapiro no rechaza la hipótesis de normalidad (al 5% de significancia), por lo que podemos asumir dicha distribucion (p-value = 0.1244 no es menor a 0.05)

## Preguntas de interés

Con el análisis previo, podemos asumir que los log-retornos del precio ajustado de las acciones en el período de observación entre 2008-11-01 y 2008-10-31 , tienen una distribución normal, con $\mu$ y $\sigma$ igual  sus estimadores muestrales (media y desv.est muestral).

**a)** Suponga que usted es inversionista y decide no vender sus acciones si la variación entre un día respecto al anterior es menor a -2%. Es decir,  $\dfrac{P_t - P_{t-1}}{P_{t-1}} < -0.02$,donde $P_t$ representa el valor ajustado de la acción al tiempo t. ¿Cuál es la probabilidad de que dicho evento ocurra? <br>
**b)** Suponga que un día observó un precio ajustado de 40 USD, ¿cuál es la probabilidad de que al siguiente día el precio ajustado sea mayor o igual a 41 USD?<br>


In [ ]:
# a)
# Una variación del -2% respecto del día anterior, es equivalente a observar un log-retorno igual a log(1 - 0.02). Luego, bajo el supuesto de normalidad, la probabilidad  de que ocurra el evento de interés será
pnorm(log(0.98), mean = mean(aux2$log_ret), sd = sd(aux2$log_ret),lower.tail = T) # Aproximadamente en un 11% de los casos no se enderán las acciones.
#b)
pnorm(log(41/40), mean = mean(aux2$log_ret), sd = sd(aux2$log_ret),lower.tail = F) # aproximadamente un 7% de probabilidad de que dicho evento ocurra.

